In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from functions import *



import make_dataset as m_d

In [3]:
###PRECIPITAZIONI


#Questo si comporta bene
#Non è utile importare i data availability tendenzialmente
colnames = ['time', 'NO_IDEA', 'intensity(?)']


dummy=m_d.safe_import('precip')
#Problema: il primo dato vien preso come dizionario
#Converto colonne
dummy=dummy.rename(columns={'201311010000': 'time', "2383": 'NO_IDEA', "1" : 'intensity(?)'})

#Riaggiungo il primo dato
precipitation=appforth(dummy,[201311010000,2383,1])

precipitation

Index(['201311010000', '2383', '1'], dtype='object')


,time,NO_IDEA,intensity(?)
0,201311010000,2383,1
1,201311010020,9452,1
2,201311010020,8278,1
3,201311010020,6871,2
4,201311010020,5689,3
...,...,...,...
3489412,201312260610,745,1
3489413,201312260610,506,1
3489414,201312260610,389,1
3489415,201312260610,393,1


In [4]:

adm_reg=m_d.safe_import('regions')

adm_reg.plot("Shape_Area")
adm_reg.head(5)


DriverError: data\raw\Com01012013\Com01012013_WGS84.shp: No such file or directory

In [ ]:
#Convertiamo alla vera patria
adm_reg=adm_reg[adm_reg["COD_PROV"]==22]
adm_reg.plot("Shape_Area")

## Merging dei dati geografici e dei dati delle linee elettriche

In [4]:
# devo prima unire i due dataset di line set -> nov e poi nov -> dicembre

#Problema: il primo dato vien preso come dizionario
#Converto colonne
temp = m_d.safe_import('SET-2')
temp=temp.rename(columns={'DG1000420': 'LINESET', "2013-12-01 00:00": 'Timestamp', "36.719997" : 'Value Amp'})
#Riaggiungo il primo dato
electro=appforth(temp,['DG1000420','2013-12-01 00:00',36.719997])

temp1 = m_d.safe_import('SET-2')
temp1=temp1.rename(columns={'DG1000420': 'LINESET', "2013-11-01 00:00": 'Timestamp', "37.439999" : 'Value Amp'})
#Riaggiungo il primo dato
electro_1=appforth(temp1,['DG1000420','2013-12-01 00:00',36.719997])

electro.append(electro_1)

electro
#del temp
#del temp1
#del electro_1
#
lines = m_d.safe_import('SET-lines')
electro = electro.merge(right=lines, how='outer')
# I change the SQUARE ID name in order to make it compatible with the geospatial data
electro = electro.rename(columns={'SQUAREID' : 'cellId'})


#Ci sono incompatibilità tra i record salvati quindi li droppo cattivo
electro.dropna()

Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['SQUAREID', 'LINESET', 'NR_UBICAZIONI'], dtype='object')


,LINESET,Timestamp,Value Amp,cellId,NR_UBICAZIONI
0,DG1000420,2013-12-01 00:00,36.719997,4037,2
1,DG1000420,2013-12-01 00:00,36.719997,4154,13
2,DG1000420,2013-12-01 00:00,36.719997,4155,20
3,DG1000420,2013-12-01 00:00,36.719997,4156,2
4,DG1000420,2013-12-01 00:00,36.719997,4269,1
...,...,...,...,...,...
11298379,DG1056626,2013-12-31 23:50,75.775002,8502,1
11298380,DG1056626,2013-12-31 23:50,75.775002,8503,4
11298381,DG1056626,2013-12-31 23:50,75.775002,8621,1
11298382,DG1056626,2013-12-31 23:50,75.775002,8972,25


In [ ]:
#GRID
grid=m_d.safe_import("grid")
grid_electro = grid.merge(electro, on = "cellId", how="inner")
print(grid_electro)
grid_electro.plot('NR_UBICAZIONI')

Index(['cellId', 'geometry'], dtype='object')
          cellId                                           geometry  \
0            155  POLYGON ((10.91493 45.69100, 10.92777 45.69079...   
1            155  POLYGON ((10.91493 45.69100, 10.92777 45.69079...   
2            155  POLYGON ((10.91493 45.69100, 10.92777 45.69079...   
3            155  POLYGON ((10.91493 45.69100, 10.92777 45.69079...   
4            155  POLYGON ((10.91493 45.69100, 10.92777 45.69079...   
...          ...                                                ...   
11298423   11099  POLYGON ((11.76477 46.51064, 11.77778 46.51033...   
11298424   11099  POLYGON ((11.76477 46.51064, 11.77778 46.51033...   
11298425   11099  POLYGON ((11.76477 46.51064, 11.77778 46.51033...   
11298426   11099  POLYGON ((11.76477 46.51064, 11.77778 46.51033...   
11298427   11099  POLYGON ((11.76477 46.51064, 11.77778 46.51033...   

            LINESET         Timestamp  Value Amp  NR_UBICAZIONI  
0         DG1031221  2013-12-01 00: